In [1]:
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import bokeh

In [ ]:
df=pd.read_csv(r"Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv")

In [ ]:
#df=df[df.Category.isin(focuscrimes)]
df["Date"] = pd.to_datetime(df["Date"])
df["Time"] = pd.to_datetime(df["Time"])
df['year']=df['Date'].dt.year.astype(str)
df['month']=df['Date'].dt.month
df['dayy']=df['Date'].dt.day
df['day'] = df['Date'].dt.weekday
df['hour'] = df['Time'].dt.hour
df['week_hour']=(df['day'])*24+(df['hour'])
cut_off=datetime.datetime(2018, 1, 1, 0, 0)
cut_off2=datetime.datetime(2010, 1, 1, 0, 0)
df=df[(df['Date']<cut_off) &(df['Date']>cut_off2)]
df_count = df.groupby(['Category', 'year'])['PdId'].count().unstack().fillna(0)

In [7]:
# Import required modules
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, save, output_file
from bokeh.transform import dodge

# Set focus crimes of interest
focuscrimes = set(['BURGLARY', 'ASSAULT', 'DRUG/NARCOTIC', 'VANDALISM', 'VEHICLE THEFT'])

# Filter the dataframe to include only rows with focus crimes
df = df[df.Category.isin(focuscrimes)]

# Group the filtered data by category and year, and count the number of occurrences
df_count = df.groupby(['Category', 'year'])['PdId'].count().unstack().fillna(0)

# Get list of unique categories and years from the dataframe
categories = list(df_count.index)
years = list(df_count.columns)

# Create a dictionary to hold the data in a format that can be used by ColumnDataSource
el_dict = {}

# Convert the grouped data to a dictionary format compatible with ColumnDataSource
for index, row in df_count.T.iterrows():
    el_dict[str(index)] = list(df_count.T.loc[index].values)
el_dict['fruits'] = list(df_count.T.columns.values)

# Assign the data to variables
fruits = categories
years = years
data = el_dict

# Create a ColumnDataSource object to hold the data
source = ColumnDataSource(data=data)

# Define color palette for bars
colors = ["#8ed075", "#7dc46b", "#6cb961", "#5ca957", "#4c9c4d", "#3c8e43", "#2c8039", "#1c7230"]

# Create a figure object for the chart
p = figure(x_range=fruits, y_range=(0, df_count.values.max() + 3000),
           title="Crime Occurrences by Year for Each Category", height=350, toolbar_location=None, tools="")

# Create vertical bars for each year of data, and add them to the chart
glyphs = []
for i, year in enumerate(years):
    vbar = p.vbar(x=dodge('fruits', -0.25 + i*0.10, range=p.x_range), top=year, source=source,
                  width=0.12, color=colors[i], legend_label=year)
    glyphs.append(vbar)

# Hide all bars except the first one initially
for i in range(1, int(len(glyphs))):
    glyphs[i].visible = False

# Set chart properties
p.x_range.range_padding = 0.1
p.yaxis.axis_label = "No. of yearly cases"
p.xaxis.axis_label = "Crime Category"
p.xgrid.grid_line_color = None
p.legend.location = "top_right"
p.legend.orientation = "vertical"
p.legend.click_policy = "hide"
p.add_layout(p.legend[0], 'right')

# Show the chart in a new tab of the default web browser
p.xaxis.major_label_text_font_size = '6pt'
show(p)
p.sizing_mode = 'fixed'
p.width = 400
p.height = 300
# Save the chart as an HTML file
output_file('bokehchart.html')


In [7]:
# Import the required libraries
import datetime
import folium
from folium.plugins import HeatMapWithTime
geo={"lat": 37.773972, "lon": -122.431297}


# Set the date range to filter the data
cut_off = datetime.datetime(2017, 4, 11, 0, 0)
cut_off2 = datetime.datetime(2017, 4, 3, 0, 0)

# Filter the dataframe to get the required data for heat map
heat_df = df[(df['Category'] == 'DRUG/NARCOTIC') & (df['Date'] < cut_off) & (df['Date'] > cut_off2)]

# Add a column for count
heat_df['count'] = 1

# Create a list of day-wise data
df_day_list = []

# Format the dates and create a list of unique dates
heat_df["DateFormatted"] = heat_df["Date"].dt.strftime("%B %-d%a")
heat_df['Date'] = heat_df['Date'].astype(str)
dates = list(set(heat_df["DateFormatted"]))
dates_sorted = sorted(dates, key=lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
date_objs = [datetime.datetime.strptime(date, "%Y-%m-%d %H:%M:%S") for date in dates_sorted]
indexx = [date_obj.strftime("%A %d. %B %Y") for date_obj in date_objs]

# Group the data by day, location and count
for day in heat_df.dayy.sort_values().unique():
    df_day_list.append(heat_df.loc[heat_df.dayy == day, ['Y', 'X', 'count']].groupby(['Y', 'X']).sum().reset_index().values.tolist())

# Create a folium map object
map_hooray = folium.Map(location=[geo['lat'], geo['lon']], zoom_start=12, tiles="Stamen Toner")

# Convert the latitude and longitude values to floats
heat_df['Y'] = heat_df['Y'].astype(float)
heat_df['X'] = heat_df['X'].astype(float)

# Filter the data for NaN values and create a list of location data
heat_df = heat_df[['Y', 'X', 'dayy']]
heat_df = heat_df.dropna(axis=0, subset=['Y', 'X', 'dayy'])
heat_data = [[row['Y'], row['X']] for index, row in heat_df.iterrows()]

# Add the heat map with time to the folium map object
HeatMapWithTime(df_day_list, name='test', auto_play=False, index=indexx, max_opacity=1).add_to(map_hooray)

# Save the map to a file
map_hooray.save('my_map.html')


C:\Users\paulo.gonzalez.isaurralde@sap.com\AppData\Local\Temp\ipykernel_20144\3989500913.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  heat_df['count'] = 1
C:\Users\paulo.gonzalez.isaurralde@sap.com\AppData\Local\Temp\ipykernel_20144\3989500913.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  heat_df["DateFormatted"] = heat_df["Date"].dt.strftime("%B %-d%a")
C:\Users\paulo.gonzalez.isaurralde@sap.com\AppData\Local\Temp\ipykernel_20144\3989500913.py:23: SettingWithCopyWarning: 
A value is trying t